#  Chatbots de Perguntas e Respostas com Llama 3 + RAG para Respostas Precisas

### 📜 **Resumo**  
Este projeto implementa um **sistema de Perguntas e Respostas baseado em IA**, utilizando o **modelo Llama 3 8B** da Meta, combinado com **RAG (Retrieval-Augmented Generation)** para buscar e recuperar informações antes de gerar respostas.  

A pipeline inclui:  
✅ **Teste sem RAG**: O modelo responde apenas com base no conhecimento embutido nele.  
✅ **Teste com RAG**: O modelo primeiro busca informações em um banco vetorial (**ChromaDB**) e usa esse contexto para gerar respostas mais precisas.  

O sistema permite consultas inteligentes com suporte a **quantização 4-bit**, tornando a execução mais leve em GPUs.  

### **Ficha Técnica**  

|🔍 **Item**|📄 **Descrição**|
|---|---|
|**🛠️ Tecnologias**|Python, PyTorch, Hugging Face Transformers, LangChain, ChromaDB|
|**📦 Dependências**|transformers, torch, bitsandbytes, accelerate, langchain, langchain_community, langchain_huggingface, langchain_chroma|
|**📌 Modelo Utilizado**|meta-llama/Meta-Llama-3-8B-Instruct|
|**📉 Quantização**|4-bit (BitsAndBytesConfig) para eficiência de memória|
|**📚 Método de Busca**|RAG (Retrieval-Augmented Generation) com banco vetorial|
|**📂 Armazenamento**|ChromaDB para indexação e recuperação de documentos|
|**🔑 Autenticação**|Token da Hugging Face via `getpass`|
|**🌍 Fonte de Dados**|Web scraping de artigos (exemplo: BBC News)|

## Instalação e Importação das Bibliotecas


In [1]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

In [2]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### Solicita o token da Hugging Face e armazena na variável de ambiente


In [3]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


## Carregamento do Modelo LLM


In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0


## Definição do Template de Prompt

In [5]:
# PHI 3
#template = """
#<|system|>
#Você é um assistente virtual prestativo e está respondendo perguntas gerais. <|end|>
#<|user|>
#{pergunta}<|end|>
#<|assistant|>
#"""

# LLAMA 3
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{pergunta}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{pergunta}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [6]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{pergunta}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

## Prompt para RAG



In [7]:
template_rag = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Pergunta: {pergunta}
Contexto: {contexto}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [8]:
template_rag

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [9]:
prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['contexto', 'pergunta'] input_types={} partial_variables={} template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'


## Aplicação para RAG com contextos maiores


## Coleta de Dados de Web


In [10]:
# importar pacotes necessários
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [11]:
loader = WebBaseLoader(web_paths = ("https://www.bbc.com/portuguese/articles/cd19vexw0y1o",),)
docs = loader.load()

In [12]:
len(docs[0].page_content)

12061

In [13]:
docs

[Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o', 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil', 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.", 'language': 'pt-br'}, page_content="Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsOscar 2024: confira todos os ganhadores dos prêmios da Academia de HollywoodCrédito, Getty ImagesLegenda da foto, Robert Downey Jr., Da'Vine Hoy Randolph, Emma Stone e Cillian Murphy com suas respectivas estatuetas do OscarArticle informationAuthor, Leire VentasRole,  Correspondente da BBC News Mundo em Los AngelesTwitter, @leire_ventas11 ma

In [14]:
print(docs[0].page_content[:300])

Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosPodcastsOscar 2024: confira tod


## Divisão do texto em partes (split)


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
splits = text_splitter.split_documents(docs)

In [17]:
len(splits)

15

In [16]:
splits[1]

Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o', 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil', 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.", 'language': 'pt-br', 'start_index': 801}, page_content="de melhor filme, melhor diretor para Christopher Nolan, melhor ator para Cillian Murphy e melhor ator coadjuvante para Robert Downey Jr.Em uma cerimônia com poucas surpresas, na qual a maior parte das apostas da crítica e do público se concretizaram, Pobres Criaturas também conquistou vários prêmios.Uma das quatro estatuetas que o filme do grego Yorgos Lanthimos arrebatou foi graças a Emma Stone, que ganhou o prêmio de melhor atriz por interpretar a protagonista Bella Baxter.O Oscar de Da'Vine Joy Randolph como melhor atriz coadjuvante foi o único de Os Rejeitados; e o de Billie Eislish, que re

In [18]:
splits[1].metadata

{'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o',
 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil',
 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.",
 'language': 'pt-br',
 'start_index': 801}

## Geração de Embeddings

In [19]:
hf_embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
input_test = "Um teste apenas"
result = hf_embeddings.embed_query(input_test)

In [21]:
len(result)

768

In [22]:
print(result)

[-0.026895053684711456, -0.011064697988331318, -0.04531998932361603, -0.0013973648892715573, 0.042436011135578156, -0.014201739802956581, 0.023354941979050636, 0.06011558324098587, 0.061521951109170914, 0.006502094678580761, 0.008159150369465351, -0.030567463487386703, 0.002060323255136609, 0.01296155247837305, 0.004251229576766491, 0.0036631699185818434, -0.026755528524518013, 0.029737044125795364, -0.009770004078745842, -0.046504564583301544, -0.028108958154916763, 0.00016858700837474316, -0.024811016395688057, -0.011828277260065079, 0.08182830363512039, 0.0014993291115388274, 0.013264901004731655, -0.06242178753018379, -0.0012287781573832035, 0.02897716872394085, -0.029528986662626266, -0.025696074590086937, 0.003377106273546815, -0.028868991881608963, 1.518518047305406e-06, -0.03882475197315216, -0.019842026755213737, -0.01765800639986992, -0.008112344890832901, -0.02513054944574833, 0.025749174878001213, 0.11367511004209518, -0.006133120507001877, -0.017986472696065903, -0.0454115

## Armazenamento no banco de dados vetorial

In [23]:
vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)

# caso fossemos usar os embeddings da Open AI, basta mudar o método, passando direto conforme abaixo
#vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

## Configuração do retriever (recuperados de texto)

In [24]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

## Geração

In [25]:
template_rag

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [ ]:
#outra opção é importar o template de um prompt de RAG via hub.pull
#prompt = hub.pull("rlm/rag-prompt")

## Construção do Prompt

In [26]:
prompt_rag = PromptTemplate(
    input_variables=["contexto", "pergunta"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['contexto', 'pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

## Formatação de Documentos

In [27]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

## Definição da Chain

In [28]:
chain_rag = ({"contexto": retriever | format_docs, "pergunta": RunnablePassthrough()}
             | prompt_rag
             | llm
             | StrOutputParser())

## Geração

### Teste sem RAG

In [30]:
# Teste sem RAG
chain = prompt | llm #i
chain.invoke("Qual filme ganhou mais oscars na premiação de 2024?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Infelizmente, a premiação dos Oscars de 2024 ainda não ocorreu. A cerimônia anual dos Academy Awards é realizada todos os anos em fevereiro ou março, e o resultado do concurso é divulgado nessa época. Por isso, não há um filme que tenha ganhado mais Oscars na premiação de 2024, pois essa premiação ainda não ocorreu.'

### Teste com RAG

In [31]:
# Teste com RAG
chain_rag.invoke("Qual filme ganhou mais oscars na premiação de 2024?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'O filme "Oppenheimer" ganhou sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor para Christopher Nolan, melhor ator para Cillian Murphy e melhor ator coadjuvante para Robert Downey Jr.'

In [34]:
chain_rag.invoke("Quem ganhou o oscar de melhor ator?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'A pergunta é: Quem ganhou o Oscar de melhor ator?\n\nA resposta é: Cillian Murphy. Ele ganhou o prêmio de melhor ator por sua interpretação de J. Robert Oppenheimer, o cientista-chefe do Projeto Manhattan, no filme "Oppenheimer".'

## Limpando o vector stores

In [ ]:
vectorstore.delete_collection()